<a href="https://colab.research.google.com/github/scope-lab-vu/transactive-blockchain/blob/master/TCPS_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To make this notebook work in Colab, the following must be done:

  1. The Cplex binary installer needs to be uploaded to your Google Drive in a directory named 'ColabLibraries'
  2. Prosumer offer data should be uploaded into a directory named 'profiles'

Mount Google Drive to access installation files

In [36]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Install cplex from Google Drive. installer.properties included in directory for silent install

In [4]:
! chmod u+x /content/gdrive/My\ Drive/ColabLibraries/cplex_studio129.linux-x86-64.bin

!/content/gdrive/My\ Drive/ColabLibraries/cplex_studio129.linux-x86-64.bin -f "installer.properties"


Preparing to install
Extracting the JRE from the installer archive...
Unpacking the JRE...
Extracting the installation resources from the installer archive...
Configuring the installer for this system's environment...

Launching installer...



Install cplex python binaries

In [0]:
%%capture
! cd /opt/ibm/ILOG/CPLEX_Studio129/python/; python3 setup.py install;

Clone transax repo

In [6]:
from getpass import getpass
import os

user = getpass('GitHub User')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password

!git clone https://$GITHUB_AUTH@github.com/scope-lab-vu/transactive-blockchain.git


GitHub User··········
GitHub password··········
Cloning into 'transactive-blockchain'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 4765 (delta 12), reused 4 (delta 1), pack-reused 4740
Receiving objects: 100% (4765/4765), 139.49 MiB | 29.40 MiB/s, done.
Resolving deltas: 100% (2180/2180), done.
Checking out files: 100% (2097/2097), done.


Install transax library

In [0]:
%%capture
! cd /content/transactive-blockchain/transax; sudo pip3 install .

Set up the microgrid and the solver

In [0]:
import cplex
from transax.Microgrid import Microgrid
from transax.MatchingSolver import Offer, MatchingSolver
microgrid = Microgrid(interval_length=1.0, C_ext=20.0, C_int=25.0, feeders=[0], prosumer_feeder={
    101: 0,
    102: 0,
    103: 0,
    104: 0,
    105: 0,
    201: 0,
    202: 0,
    203: 0,
    204: 0,
    207: 0,
    301: 0,
    302: 0,
    303: 0,
    304: 0,
    305: 0,
  })
solver = MatchingSolver(microgrid)

Build a dictionary of all the prosumer offers

In [0]:
import pandas as pd
import numpy as np
prosumer_df = {}
directory = os.fsencode('/content/gdrive/My Drive/profiles')
for file in os.listdir(directory):
  filename = os.fsdecode(file)
  prosumer_id = int(filename.split('_')[1].split('.')[0])
  prosumer_df[prosumer_id] = pd.read_csv('/content/gdrive/My Drive/profiles/' + filename)


Create lists for buying and selling offers

In [0]:
buying_offers = []
selling_offers = []
for prosumer in prosumer_df.values():
  for row in prosumer.iterrows():
      startTime = int(row[1]['startTime'])
      endTime = int(row[1]['endTime'])
      energy = float(row[1]['energy'])
      offer = Offer(prosumer_id, prosumer_id, startTime, endTime, np.abs(energy), 1)
      if energy > 0:
        buying_offers.append(offer)
      elif energy < 0:
        selling_offers.append(offer)

In [112]:
(trades, objective) = solver.solve(buying_offers=buying_offers, selling_offers=selling_offers)
print("Success: {} energy traded".format(objective))

Success: 762.9103701513632 energy traded
